In [1]:
import pybullet as p
import numpy as np

pybullet build time: Jan 29 2025 23:16:28


In [2]:
# start simulation
physClient = p.connect(p.GUI)  # pybullet.DIRECT for non-GUI mode

In [3]:
import pybullet_data
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())

plane = p.loadURDF("plane.urdf")
# reset simulation
p.setGravity(0, 0, -9.81)
robot = p.loadURDF("newton/newton.urdf", [0, 0, 0.33])
print(robot)

1


In [4]:
foot_links = ['FL_FOOT', 'FR_FOOT', 'HL_FOOT', 'HR_FOOT']
num_joints = p.getNumJoints(robot)
foot_indices = []

for i in range(num_joints):
    joint_info = p.getJointInfo(robot, i)
    print(type(joint_info))
    print(f"Joint {i}: {joint_info[1].decode('utf-8')}")

    for  value in joint_info:
        print(value)
    print(f"Joint {i}: {joint_info}")
    if joint_info[1].decode('utf-8') in foot_links:
        foot_indices.append(i)

print("Foot link indices:", foot_indices)

<class 'tuple'>
Joint 0: FL_HAA
0
b'FL_HAA'
0
7
6
1
0.0
0.0
-10.0
10.0
1000.0
1000.0
b'FL_SHOULDER'
(1.0, 0.0, 0.0)
(0.21492966000000002, 0.08749895, 0.0006021)
(0.0, 0.0, 0.0, 1.0)
-1
Joint 0: (0, b'FL_HAA', 0, 7, 6, 1, 0.0, 0.0, -10.0, 10.0, 1000.0, 1000.0, b'FL_SHOULDER', (1.0, 0.0, 0.0), (0.21492966000000002, 0.08749895, 0.0006021), (0.0, 0.0, 0.0, 1.0), -1)
<class 'tuple'>
Joint 1: FL_HFE
1
b'FL_HFE'
0
8
7
1
0.0
0.0
-10.0
10.0
1000.0
1000.0
b'FL_UPPER_LEG'
(0.0, 1.0, 0.0)
(0.07457251, 0.01838935, -1.384e-05)
(0.0, 0.0, 0.0, 1.0)
0
Joint 1: (1, b'FL_HFE', 0, 8, 7, 1, 0.0, 0.0, -10.0, 10.0, 1000.0, 1000.0, b'FL_UPPER_LEG', (0.0, 1.0, 0.0), (0.07457251, 0.01838935, -1.384e-05), (0.0, 0.0, 0.0, 1.0), 0)
<class 'tuple'>
Joint 2: FL_KFE
2
b'FL_KFE'
0
9
8
1
0.0
0.0
-10.0
10.0
1000.0
1000.0
b'FL_LOWER_LEG'
(0.0, 1.0, 0.0)
(-1.53e-05, 0.019773600000000006, -0.0816177)
(0.0, 0.0, 0.0, 1.0)
1
Joint 2: (2, b'FL_KFE', 0, 9, 8, 1, 0.0, 0.0, -10.0, 10.0, 1000.0, 1000.0, b'FL_LOWER_LEG', (0.0, 1.

In [5]:
print("\n=== Joint Information for Solo12 Quadruped ===")
num_joints = p.getNumJoints(robot)
joint_info = []

for i in range(num_joints):
    info = p.getJointInfo(robot, i)
    joint_info.append(info)
    print(f"\nJoint {i}:")
    print(f"  Name: {info[1].decode('utf-8')}")
    print(f"  Type: {info[2]}")  # 0=revolute, 1=prismatic, 2=spherical, 3=planar, 4=fixed
    print(f"  Lower limit: {info[8]}")
    print(f"  Upper limit: {info[9]}")
    print(f"  Max force: {info[10]}")
    print(f"  Max velocity: {info[11]}")
    
    # Get current joint state
    state = p.getJointState(robot, i)
    print(f"  Current position: {state[0]}")
    print(f"  Current velocity: {state[1]}")
    print(f"  Current reaction forces: {state[2]}")


=== Joint Information for Solo12 Quadruped ===

Joint 0:
  Name: FL_HAA
  Type: 0
  Lower limit: -10.0
  Upper limit: 10.0
  Max force: 1000.0
  Max velocity: 1000.0
  Current position: 0.0
  Current velocity: 0.0
  Current reaction forces: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

Joint 1:
  Name: FL_HFE
  Type: 0
  Lower limit: -10.0
  Upper limit: 10.0
  Max force: 1000.0
  Max velocity: 1000.0
  Current position: 0.0
  Current velocity: 0.0
  Current reaction forces: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

Joint 2:
  Name: FL_KFE
  Type: 0
  Lower limit: -10.0
  Upper limit: 10.0
  Max force: 1000.0
  Max velocity: 1000.0
  Current position: 0.0
  Current velocity: 0.0
  Current reaction forces: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

Joint 3:
  Name: FL_ANKLE
  Type: 4
  Lower limit: -10.0
  Upper limit: 10.0
  Max force: 1000.0
  Max velocity: 1000.0
  Current position: 0.0
  Current velocity: 0.0
  Current reaction forces: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

Joint 4:
  Name: FR_HAA
  Type: 0
  Lower limit: 

In [6]:
p.setPhysicsEngineParameter(numSolverIterations=100)
p.setPhysicsEngineParameter(enableConeFriction=1)

In [7]:
# Get number of joints and their info
num_joints = p.getNumJoints(robot)
print(f"Number of joints: {num_joints}")

# Enable force/torque sensors for all joints
for i in range(num_joints):
    p.enableJointForceTorqueSensor(robot, i, enableSensor=1)
    joint_info = p.getJointInfo(robot, i)
    print(f"Joint {i}: {joint_info[1]}")

Number of joints: 16
Joint 0: b'FL_HAA'
Joint 1: b'FL_HFE'
Joint 2: b'FL_KFE'
Joint 3: b'FL_ANKLE'
Joint 4: b'FR_HAA'
Joint 5: b'FR_HFE'
Joint 6: b'FR_KFE'
Joint 7: b'FR_ANKLE'
Joint 8: b'HL_HAA'
Joint 9: b'HL_HFE'
Joint 10: b'HL_KFE'
Joint 11: b'HL_ANKLE'
Joint 12: b'HR_HAA'
Joint 13: b'HR_HFE'
Joint 14: b'HR_KFE'
Joint 15: b'HR_ANKLE'


In [8]:
FL_joints = [0, 1, 2]    # Front Left: HAA, HFE, KFE
FR_joints = [4, 5, 6]    # Front Right: HAA, HFE, KFE 
HL_joints = [8, 9, 10]   # Hind Left: HAA, HFE, KFE
HR_joints = [12, 13, 14] # Hind Right: HAA, HFE, KFE


In [9]:
standing_pose = {
    "FL": [0.0, 0.5, -1.0],  # HAA, HFE, KFE angles in radians
    "FR": [0.0, 0.5, -1.0],
    "HL": [0.0, 0.5, -1.0],
    "HR": [0.0, 0.5, -1.0]
}

In [12]:
def set_standing_pose():
    # Front Left leg
    for joint, angle in zip(FL_joints, standing_pose["FL"]):
        p.resetJointState(robot, joint, angle)
        p.setJointMotorControl2(robot, joint, 
                              p.POSITION_CONTROL,
                              targetPosition=angle,
                              force=1000)
    
    # Front Right leg
    for joint, angle in zip(FR_joints, standing_pose["FR"]):
        p.resetJointState(robot, joint, angle)
        p.setJointMotorControl2(robot, joint, 
                              p.POSITION_CONTROL,
                              targetPosition=angle,
                              force=1000)
    
    # Hind Left leg
    for joint, angle in zip(HL_joints, standing_pose["HL"]):
        p.resetJointState(robot, joint, angle)
        p.setJointMotorControl2(robot, joint, 
                              p.POSITION_CONTROL,
                              targetPosition=angle,
                              force=1000)
    
    # Hind Right leg
    for joint, angle in zip(HR_joints, standing_pose["HR"]):
        p.resetJointState(robot, joint, angle)
        p.setJointMotorControl2(robot, joint, 
                              p.POSITION_CONTROL,
                              targetPosition=angle,
                              force=1000)

In [13]:
set_standing_pose()
for _ in range(100):
    p.stepSimulation()

In [14]:
import time
def print_joint_torques():
    print("\n=== Joint Torques Analysis ===")
    
    # Format string for header and rows
    header_format = "{:<8} | {:<10} | {:<10} | {:<10}"
    row_format = "{:<8} | {:10.3f} | {:10.3f} | {:10.3f}"
    
    # Print header
    print("\n" + header_format.format("Leg", "HAA (N·m)", "HFE (N·m)", "KFE (N·m)"))
    print("-" * 45)  # Separator line
    
    # Get joint torques for each leg
    legs = ["FL", "FR", "HL", "HR"]
    joint_indices = {
        "FL": [0, 1, 2],     # Front Left
        "FR": [4, 5, 6],     # Front Right
        "HL": [8, 9, 10],    # Hind Left
        "HR": [12, 13, 14]   # Hind Right
    }
    
    # Print torques for each leg
    for leg in legs:
        indices = joint_indices[leg]
        torques = []
        for idx in indices:
            joint_state = p.getJointState(robot, idx)
            torques.append(joint_state[3])  # index 3 is the applied torque
            
        print(row_format.format(leg, torques[0], torques[1], torques[2]))

# Update simulation and print torques
try:
    while True:
        print_joint_torques()
        p.stepSimulation()
        time.sleep(0.5)  # Slower update for readability
except KeyboardInterrupt:
    print("\nVisualization stopped by user")


=== Joint Torques Analysis ===

Leg      | HAA (N·m)  | HFE (N·m)  | KFE (N·m) 
---------------------------------------------
FL       |     -0.024 |     -0.429 |      0.148
FR       |      0.097 |     -0.534 |      0.140
HL       |      0.256 |      0.503 |      0.747
HR       |     -0.314 |      0.687 |      0.782

=== Joint Torques Analysis ===

Leg      | HAA (N·m)  | HFE (N·m)  | KFE (N·m) 
---------------------------------------------
FL       |     -0.022 |     -0.429 |      0.148
FR       |      0.098 |     -0.534 |      0.140
HL       |      0.258 |      0.502 |      0.746
HR       |     -0.317 |      0.688 |      0.783

=== Joint Torques Analysis ===

Leg      | HAA (N·m)  | HFE (N·m)  | KFE (N·m) 
---------------------------------------------
FL       |     -0.021 |     -0.429 |      0.147
FR       |      0.098 |     -0.535 |      0.140
HL       |      0.260 |      0.501 |      0.746
HR       |     -0.321 |      0.690 |      0.783

=== Joint Torques Analysis ===

Leg      |